# Hi! You can find a live demo of the app [here](https://huggingface.co/spaces/Epitech/hand-sign-detection).

### Information
As of today, there are about 70 million deaf people in the world and they collectively use more than 300 different sign languages.
This software will allow people using sign language to communicate with people who do not know it.

### How does it work?
Using a camera (for example the one on your phone) film a person, and let our IA transcript the sign on your screen.
*For now, only the words: **eat**, **drink**, **watch** and **ok** are recognizable by our software.* 

### Who'll use it?
Any persons who wich to communicate with someone speaking sign language.
This could happen in lot of scenarios like when an hearing impaired wants to speak with:
- a merchant
- a doctor
- a police 

### What technologies are behind:
For training:
- Mediapipe
- OpenCV
- Teachable

For the IA:
- Keras
- Tensorflow

For the code base:
- Python

For the app:
- Gradio

1. First, please clone the following repository to have access to the necessary files.

In [ ]:
! git clone --branch notebook https://github.com/n0no123/hand-sign-detection/

2. Install the required dependencies.

In [ ]:
pip install -r '/content/hand-sign-detection/requirements.txt'

3. Import our libraries.

In [ ]:
import cv2
import gradio as gr
import math
import numpy as np
from cvzone.ClassificationModule import Classifier
from cvzone.HandTrackingModule import HandDetector

4. Set our global variables.

In [ ]:
bgSize = 96
classifier = Classifier('/content/hand-sign-detection/keras_model.h5', '/content/hand-sign-detection/labels.txt')
detector = HandDetector(maxHands=1)
labels = ['Look', 'Drink', 'Eat', 'Ok']
offset = 20

5. Set the frame analysis function.

In [ ]:
def segment(image):
    hands, frame = detector.findHands(image)
    try:
        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']
            croppedHand = np.ones((bgSize, bgSize, 3), np.uint8) * 12
            imgCrop = frame[y - offset:y + h +
                            offset, x - offset:x + w + offset]
            aspectRatio = h / w
            if aspectRatio > 1:
                constant = bgSize / h
                wComputed = math.floor(constant * w)
                bgResize = cv2.resize(imgCrop, (wComputed, bgSize))
                bgResizeShape = bgResize.shape
                wGap = math.floor((bgSize-wComputed)/2)
                croppedHand[:bgResizeShape[0],
                            wGap:wGap + wComputed] = bgResize
            else:
                constant = bgSize / w
                hComputed = math.floor(constant * h)
                bgResize = cv2.resize(imgCrop, (bgSize, hComputed))
                bgResizeShape = bgResize.shape
                hGap = math.floor((bgSize - hComputed) / 2)
                croppedHand[hGap: hComputed + hGap, :] = bgResize
            _, index = classifier.getPrediction(croppedHand, draw=False)
            return labels[index]
    except Exception as e:
        print(e)
    return 'No sign detected'

6. Create the Gradio interface. *To have a better experience, please use the live [demo]((https://huggingface.co/spaces/Epitech/hand-sign-detection).)*

In [ ]:
gr.interface.Interface(fn=segment, live=True, inputs=gr.Image(source='webcam', streaming=True), outputs='text').launch()